In [1]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from category_encoders import BinaryEncoder
from sklearn.decomposition import PCA
from sklearn import set_config
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import GridSearchCV, cross_val_score

In [2]:
df=pd.read_csv("data//clean//vehiculos.csv", index_col="Unnamed: 0")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14631 entries, 0 to 14657
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Marca                   14631 non-null  object 
 1   Modelo                  14631 non-null  object 
 2   Precio                  14631 non-null  float64
 3   Potencia                14631 non-null  float64
 4   Tipo vendedor           14631 non-null  object 
 5   Categoría               14631 non-null  object 
 6   Tipo de vehículo        14631 non-null  object 
 7   puertas                 14631 non-null  int64  
 8   Versión del país        14631 non-null  object 
 9   Garantía                14631 non-null  int64  
 10  Kilometraje             14631 non-null  float64
 11  Año                     14631 non-null  int64  
 12  Tipo de cambio          14631 non-null  object 
 13  Capacidad               14631 non-null  float64
 14  Consumo de combustible  14631 non-null  flo

Como 'Versión del país' siempre contiene el valor 'España' no es relevante y lo podemos eliminar.

In [3]:
df=df.drop('Versión del país', axis=1)

Definimos las columnas numéricas y las categóricas:

In [4]:
num_cols=df.select_dtypes(['int64', 'float64']).columns
cat_cols=df.select_dtypes(['object']).columns

De las columnas numéricas hay que mirar cuales tienen distribución gausiana, para saber a cuales podemos estandarizar con el StandartScaler.

In [5]:
def gaussian_columns(df):
    gaus_cols=[]
    for column in df.columns:
        stat, p = shapiro(df[column])
        if p > 0.05:
            gaus_cols.append(column)
    return gaus_cols

In [6]:
df_num=df[num_cols]
gaus_cols=gaussian_columns(df_num)
print(gaus_cols)

[]


c:\Users\mabi1\anaconda3\envs\proj\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Ninguna es probable que tenga distribución gausiana. Ahora de las columnas numéricas vamos a separar las que tienen outliers de las que no, para ello cogeremos el dataframe con solo las columnas numéricas, tomaremos también el primer cuantil, el tercero y la diferencia entre ambos y con una fórmula matemática seleccionaremos las columnas donde hay outliers. Una vez obtenidas las columnas con outliers haremos un drop de las mismas en el dataframe de columnas numéricas y las restantes serán las que no tienen outliers. Es importante tener en cuenta cuales tienen outliers y cuales no para saber qué tipo de escalado aplicarles.

In [7]:
q1 = df_num.quantile(0.25)
q3 = df_num.quantile(0.75)
diff = q3 - q1
out_cols=df_num.columns[((df_num < (q1 - 1.5 * diff)) |(df_num > (q3 + 1.5 * diff))).any(axis=0)]
no_out_cols=df_num.drop(out_cols, axis=1).columns
#Eliminamos la variable target
out_cols=out_cols.drop('Precio')

In [8]:
print(out_cols)
print(no_out_cols)

Index(['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad',
       'Consumo de combustible', 'plazas', 'Número de marchas',
       'Número de cilindros', 'Peso'],
      dtype='object')
Index(['Mes', 'CP'], dtype='object')


Así pues, ahora tenemos las columnas de nuestro dataset divididas en tres sets: columnas categóricas (cat_cols), columnas numéricas con outliers(out_cols) y columnas numéricas sin outliers (no_out_cols).
A las columnas categóricas les aplicaremos OneHotEncoder y posteriormente BinaryEncoder, para poder inlcuirlas en el PCA, a las que tienen outliers RobustScaler y a las que no tienen outliers MinMaxScaler.

In [9]:
out_pipeline = Pipeline(steps=[
    ('scale',RobustScaler())
])

no_out_pipeline = Pipeline(steps=[
    ('scale',MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('scale', OneHotEncoder(handle_unknown='ignore', sparse=False)), # para que gestiones si en los datos de test hay una clase que no estaba en el train
    #('binary', BinaryEncoder())
])

Añadiremos estas pequeñas pipeslines a un ColumnTransformer para que a cada tipo de columna se le aplique lo indicado. Le indicaremos remainder='drop' para que ignore cualquier columna del dataframe que no esté contemplada en esas categorías. Y n_jobs= -1 para que use todos los procesadores que tengamos en paralelo.

In [10]:
col_trans = ColumnTransformer(transformers=[
    ('out_pipeline',out_pipeline,out_cols),
    ('no_out_pipeline',no_out_pipeline,no_out_cols),
    ('cat_pipeline', cat_pipeline, cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

Este column transformer será añadido a un pipeline final que también contendrá el modelo. Pero primero deberíamos decidir qué modelo utilizar.

In [11]:
pca_pipe=Pipeline(steps=[
    ('col_trans', col_trans),
    ('pca', PCA(n_components=0.95))
])

Vamos a visualizar la pipeline

In [12]:
set_config(display='diagram')
display(pca_pipe)

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('out_pipeline',
                                                  Pipeline(steps=[('scale',
                                                                   RobustScaler())]),
                                                  Index(['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad',
       'Consumo de combustible', 'plazas', 'Número de marchas',
       'Número de cilindros', 'Peso'],
      dtype='object')),
                                                 ('no_out_pipeline',
                                                  Pipeline(steps=[('scale',
                                                                   MinMaxScal...
                                                  Index(['Mes', 'CP'], dtype='object')),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('scale',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('binary',
                                                                   BinaryEncoder())]),
                                                  Index(['Marca', 'Modelo', 'Tipo vendedor', 'Categoría', 'Tipo de vehículo',
       'Tipo de cambio', 'Color exterior', 'Color original', 'Tracción',
       'Tipo de combustible', 'Ciudad', 'provincia'],
      dtype='object'))])),
                ('pca', PCA(n_components=0.95))])

No hacemos separación entre datos de train y test porque con el GridSearchCV usaremos cross validation.

In [13]:
X=df.drop('Precio', axis=1)
y=df['Precio']

In [14]:
X=pca_pipe.fit_transform(X)

Ya tenemos los datos estandarizados y con dimensionalidad reducida. Es hora de poner a prueba diferentes modelos de regresión para elegir uno. Probaremos los modelos de regresión LinearRegression, Lasso y RandomForest

Model LinearRegression

In [15]:
params={
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'positive': [True, False]
}

In [16]:
grid_search_linear=GridSearchCV(estimator=LinearRegression(), param_grid=params,scoring='r2', cv=5)
grid_search_linear.fit(X, y)

GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'positive': [True, False]},
             scoring='r2')

In [17]:
print("Los mejors parámetros para LinearRegression son:", grid_search_linear.best_params_ )
print("La puntuación R2 usando los mejores parámetros es:", grid_search_linear.best_estimator_.score(X, y))

Los mejors parámetros para LinearRegression son: {'copy_X': True, 'fit_intercept': True, 'positive': True}
La puntuación R2 usando los mejores parámetros es: 0.16333955712193693


Podemos ver como este modelo no se ajusta en absoluto a los datos, debido a su simplicidad.

Regresión Lasso

In [18]:
params={
    'alpha': list(np.arange(0.01, 0.1, 0.01)),
    'fit_intercept': [True, False],
    'precompute' : [True, False],
    'copy_X': [True, False],
    'max_iter': list(np.arange(1000,3000,1000)),
    'tol': list(np.arange(0.0001,0.0003, 0.0001)),
    'warm_start' : [True, False],
    'positive': [True, False],
    'selection' : ['cyclic', 'random']
}

In [19]:
grid_search_lasso=GridSearchCV(estimator=Lasso(), param_grid=params,scoring=['r2', 'neg_mean_squared_error'], refit='r2', cv=5)
grid_search_lasso.fit(X, y)

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0.01, 0.02, 0.03, 0.04, 0.05,
                                   0.060000000000000005, 0.06999999999999999,
                                   0.08, 0.09],
                         'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'max_iter': [1000, 2000], 'positive': [True, False],
                         'precompute': [True, False],
                         'selection': ['cyclic', 'random'],
                         'tol': [0.0001, 0.0002], 'warm_start': [True, False]},
             refit='r2', scoring=['r2', 'neg_mean_squared_error'])

In [20]:
print("Los mejors parámetros para Lasso son:", grid_search_lasso.best_params_ )
print("La puntuación R2 usando los mejores parámetros es:", grid_search_lasso.best_estimator_.score(X, y))

Los mejors parámetros para Lasso son: {'alpha': 0.08, 'copy_X': False, 'fit_intercept': True, 'max_iter': 2000, 'positive': True, 'precompute': False, 'selection': 'random', 'tol': 0.0002, 'warm_start': True}
La puntuación R2 usando los mejores parámetros es: 0.16333955712173076


Este modelo tampoco nos ha dado mejores resultados que el anterior.